Integrating multiple databases
==============================

In this example, we use ArchiTXT to integrate two databases (Northwind and Chinook).

In [ ]:
import itables

itables.init_notebook_mode(connected=True)

## Load databases

### Northwind

The Northwind sample database was provided with Microsoft Access as a tutorial schema for managing small business customers, orders, inventory, purchasing, suppliers, shipping, and employees.
Northwind is an excellent tutorial schema for a small-business ERP, with customers, orders, inventory, purchasing, suppliers, shipping, employees, and single-entry accounting.


```{mermaid}
    ---
    config:
        theme: neutral
    ---
    erDiagram
        CustomerCustomerDemo }o--|| CustomerDemographics : "has"
        CustomerCustomerDemo }o--|| Customers : "relates to"

        Employees }|--o| Employees : "reports to"
        Employees ||--o{ EmployeeTerritories : "assigned to"

        Orders }o--|| Shippers : "ships via"
        Orders }o--|| Customers : "placed by"
        Orders }o--|| Employees : "sold by"

        "Order Details" }o--|| Orders : "contain"
        "Order Details" }o--|| Products : "refer to"

        Products }o--|| Categories : "categorized in"
        Products }o--|| Suppliers : "supplied by"

        Territories }|--|| Regions : "located in"
        EmployeeTerritories }o--|| Territories : "assigned to"

        Categories {
            int     CategoryID      PK
            string  CategoryName
            string  Description
            blob    Picture
        }

        CustomerCustomerDemo {
            string  CustomerID       PK, FK
            string  CustomerTypeID   PK, FK
        }

        CustomerDemographics {
            string  CustomerTypeID   PK
            string  CustomerDesc
        }

        Customers {
            string  CustomerID       PK
            string  CompanyName
            string  ContactName
            string  ContactTitle
            string  Address
            string  City
            string  Region
            string  PostalCode
            string  Country
            string  Phone
            string  Fax
        }

        Employees {
            int     EmployeeID       PK
            string  LastName
            string  FirstName
            string  Title
            string  TitleOfCourtesy
            date    BirthDate
            date    HireDate
            string  Address
            string  City
            string  Region
            string  PostalCode
            string  Country
            string  HomePhone
            string  Extension
            blob    Photo
            string  Notes
            int     ReportsTo        FK
            string  PhotoPath
        }

        EmployeeTerritories {
            int     EmployeeID       PK, FK
            int     TerritoryID      PK, FK
        }

        "Order Details" {
            int     OrderID          PK, FK
            int     ProductID        PK, FK
            float   UnitPrice
            int     Quantity
            real    Discount
        }

        Orders {
            int     OrderID          PK
            string  CustomerID       FK
            int     EmployeeID       FK
            datetime OrderDate
            datetime RequiredDate
            datetime ShippedDate
            int     ShipVia          FK
            numeric Freight
            string  ShipName
            string  ShipAddress
            string  ShipCity
            string  ShipRegion
            string  ShipPostalCode
            string  ShipCountry
        }

        Products {
            int     ProductID        PK
            string  ProductName
            int     SupplierID       FK
            int     CategoryID       FK
            int     QuantityPerUnit
            float   UnitPrice
            int     UnitsInStock
            int     UnitsOnOrder
            int     ReorderLevel
            string  Discontinued
        }

        Regions {
            int     RegionID         PK
            string  RegionDescription
        }

        Shippers {
            int     ShipperID        PK
            string  CompanyName
            string  Phone
        }

        Suppliers {
            int     SupplierID       PK
            string  CompanyName
            string  ContactName
            string  ContactTitle
            string  Address
            string  City
            string  Region
            string  PostalCode
            string  Country
            string  Phone
            string  Fax
            string  HomePage
        }

        Territories {
            string  TerritoryID      PK
            string  TerritoryDescription
            int     RegionID         FK
        }
```

In [ ]:
import urllib.request
from pathlib import Path

db_file = Path('northwind.db')

if not db_file.exists():
    urllib.request.urlretrieve(
        'https://github.com/jpwhite3/northwind-SQLite3/raw/refs/heads/main/dist/northwind.db',
        filename=db_file,
    )

In [ ]:
from architxt.database.loader import read_sql
from sqlalchemy import create_engine

with create_engine('sqlite:///northwind.db').connect() as connection:
    northwind_forest = list(read_sql(connection, sample=20))

### Chinook

The Chinook Sample Database is a fictional digital media store database used for learning and testing SQL queries.
It is modeled after real-world online music stores like iTunes and includes data related to artists, albums, tracks, customers, invoices, employees, and genres.

```{mermaid}
    ---
    config:
        theme: neutral
    ---
    erDiagram
        Artist ||--|{ Album : "hasAuthor"

        Employee ||--|{ Customer : "supports"
        Employee ||--|{ Employee : "reportsTo"

        Customer ||--|{ Invoice : "makeOrder"
        Invoice ||--|{ InvoiceLine : "contains"
        Track ||--|{ InvoiceLine : "soldIn"

        Playlist ||--|{ PlaylistTrack : "includes"
        Track ||--|{ PlaylistTrack : "appearsIn"

        Album ||--|{ Track : "has"
        Genre ||--|{ Track : "hasGenre"
        MediaType ||--|{ Track : "encodedAs"

        Album {
            int     AlbumId   PK
            string  Title
            int     ArtistId  FK
        }
        Artist {
            int     ArtistId  PK
            string  Name
        }
        Customer {
            int     CustomerId    PK
            string  FirstName
            string  LastName
            string  Company
            string  Address
            string  City
            string  State
            string  Country
            string  PostalCode
            string  Phone
            string  Fax
            string  Email
            int     SupportRepId  FK
        }
        Employee {
            int     EmployeeId  PK
            string  LastName
            string  FirstName
            string  Title
            int     ReportsTo   FK
            date    BirthDate
            date    HireDate
            string  Address
            string  City
            string  State
            string  Country
            string  PostalCode
            string  Phone
            string  Fax
            string  Email
        }
        Genre {
            int     GenreId     PK
            string  Name
        }
        Invoice {
            int     InvoiceId           PK
            int     CustomerId          FK
            date    InvoiceDate
            string  BillingAddress
            string  BillingCity
            string  BillingState
            string  BillingCountry
            string  BillingPostalCode
            float   Total
        }
        InvoiceLine {
            int     InvoiceLineId   PK
            int     InvoiceId       FK
            int     TrackId         FK
            float   UnitPrice
            int     Quantity
        }
        MediaType {
            int     MediaTypeId PK
            string  Name
        }
        Playlist {
            int     PlaylistId  PK
            string  Name
        }
        PlaylistTrack {
            int PlaylistId  PK, FK
            int TrackId     PK, FK
        }
        Track {
            int     TrackId     PK
            string  Name
            int     AlbumId     FK
            int     MediaTypeId FK
            int     GenreId     FK
            string  Composer
            int     Milliseconds
            int     Bytes
            float   UnitPrice
        }
```

In [ ]:
db_file = Path('chinook.db')

if not db_file.exists():
    urllib.request.urlretrieve(
        'https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite',
        filename=db_file,
    )

In [ ]:
with create_engine('sqlite:///chinook.db').connect() as connection:
    chinook_forest = list(read_sql(connection, sample=10))

## Integration

We combine the two forests representing the databases and inspect the schema.

In [ ]:
from architxt.schema import Schema

forest = northwind_forest + chinook_forest
schema = Schema.from_forest(forest)
print(schema.as_cfg())

### Explore the schema

The schema is complex, and some groups like `Orders` and `Invoice` may be merged to simplify the instance.
We first need to rename semantically equivalent properties to make them uniform across both databases.

We'll use a mapping dictionary to rename entities and standardize them.

In [ ]:
from architxt.tree import NodeLabel, NodeType, has_type

entity_mapping = {
    # Track / Product
    'TrackId': 'ProductID',
    # Invoice / Order
    'InvoiceId': 'OrderID',
    'InvoiceDate': 'OrderDate',
    'ShipAddress': 'BillingAddress',
    'ShipCity': 'BillingCity',
    'ShipCountry': 'BillingCountry',
    'ShipPostalCode': 'BillingPostalCode',
    'BillingState': 'State',
    'ShipRegion': 'Region',
    # Employees / Customers
    'CompanyName': 'Company',
    'HomePhone': 'Phone',
    'CustomerId': 'CustomerID',
    'EmployeeId': 'EmployeeID',
}

for tree in forest:
    for subtree in tree.subtrees():
        if has_type(subtree, NodeType.ENT) and subtree.label.name in entity_mapping:
            subtree.label = NodeLabel(NodeType.ENT, entity_mapping[subtree.label.name])

schema = Schema.from_forest(forest)
print(schema.as_cfg())

Now that we've standardized the property names, we can visualize the initial distribution of groups in the schema.

To do this, let's count how many instances of each group exist in the schema.

In [ ]:
from collections import Counter

import pandas as pd
from architxt.tree import NodeType, has_type

group_counter = Counter(
    subtree.label.name for tree in forest for subtree in tree.subtrees() if has_type(subtree, NodeType.GROUP)
)

df_initial = pd.DataFrame.from_dict(group_counter, orient='index', columns=['Count'])
df_initial = df_initial.sort_values('Count', ascending=False)
df_initial.loc['Total'] = group_counter.total()
df_initial.index.name = 'Group'

df_initial

Next, we can use ArchiTXT's tree clustering algorithm to identify which groups can be merged.

In [ ]:
from architxt.similarity import equiv_cluster

clusters = equiv_cluster(forest, tau=0.85)

Let's visualize the clustering result in a new table.


In [ ]:
new_groups = {
    str(klass[0].label): Counter(subtree.label.name for subtree in klass if has_type(subtree, NodeType.GROUP))
    for klass in clusters
}

df_new = pd.DataFrame.from_dict(new_groups, orient='index').fillna(0).astype(int)
df_new['Total'] = df_new.sum(axis=1)
df_new = df_new.sort_values('Total', ascending=False)
df_new.index.name = 'Class'

df_new

Let's visualize the clustering result as a bar chart to better understand the distribution of groups across equivalent classes.

In [ ]:
import plotly.express as px

fig = px.bar(
    df_new.drop(columns=['Total']),
    labels={'value': 'Count', 'variable': 'Group', 'index': 'Class'},
    title='Groups repartition per Equivalent Class',
)

fig.update_layout(barmode='stack', xaxis_title='Equivalent Class', yaxis_title='Count', xaxis_tickangle=-45)

### Simplification

Now that we saw potential groupings, we can use ArchiTXT to perform simplification and rewrite the forest into a condensed version.

In [ ]:
from architxt.simplification.tree_rewriting import rewrite

rewrite(forest, tau=0.85, min_support=100, epoch=10, edit_ops=[])

In [ ]:
new_schema = Schema.from_forest(forest)
print(new_schema.as_cfg())

In [ ]:
all_datasets = new_schema.extract_datasets(forest)
group, dataset = max(all_datasets.items(), key=lambda x: len(x[1]))

print(f'Group: {group}')

dataset